In [1]:
import numpy as np
import pandas as pd
import random


pygame 2.5.2 (SDL 2.28.3, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:

BOW_train = pd.read_csv('../project_data/data/bag-of-words/bow.train.csv').replace({'label': {0: -1}})
BOW_test = pd.read_csv('../project_data/data/bag-of-words/bow.test.csv').replace({'label': {0: -1}})
BOW_eval = pd.read_csv('../project_data/data/bag-of-words/bow.eval.anon.csv').replace({'label': {0: -1}})

In [3]:
glove_train = pd.read_csv('../project_data/data/glove/glove.train.csv').replace({'label': {0: -1}})
glove_test = pd.read_csv('../project_data/data/glove/glove.test.csv').replace({'label': {0: -1}})
glove_eval = pd.read_csv('../project_data/data/glove/glove.eval.anon.csv').replace({'label': {0: -1}})

In [4]:
tfidf_train = pd.read_csv('../project_data/data/tfidf/tfidf.train.csv').replace({'label': {0: -1}})    
tfidf_test = pd.read_csv('../project_data/data/tfidf/tfidf.test.csv').replace({'label': {0: -1}})
tfidf_eval = pd.read_csv('../project_data/data/tfidf/tfidf.eval.anon.csv').replace({'label': {0: -1}})


In [6]:
def stochastic_sub_gradient_descent_SVM(train_data, train_labels, LR_0, C, T, SVM_obj_threshold) :
    N = len(train_data)
    w = np.zeros(len(train_data[0]))
    LR_t = LR_0
    prev_SVM_obj = None
    for t in range(T):
        # shuffle the data
        indices = list(range(N))
        random.shuffle(indices) ### this may not work, 
        # calculate the learning rate
        LR_t = LR_0/(1+t)
        for i in range(N):
            yi = train_labels[i]
            xi = train_data[i]
            if yi * np.dot(w, xi) <= 1:
                w = (1-LR_t)*w + LR_t*C*(yi * xi)
            else:
                w = (1-LR_t)*w
        # calculate SVM objective after each epoch, may have to sum but idk
        SVM_obj = np.sum(np.maximum(0, 1 - train_labels * np.dot(train_data, w))) + 0.5 * C * np.dot(w, w)
        # may not even need this part
        if prev_SVM_obj == None:
            prev_SVM_obj = SVM_obj
            continue
        # stop iterating if change in SVM objective is smaller than threshold
        if prev_SVM_obj - SVM_obj < SVM_obj_threshold:
            break
        prev_SVM_obj = SVM_obj
    return w


In [7]:
## modifying this from assignment to use accuracy instead of F_1

def CV (train_folds, C, LR_0, T, SVM_obj_threshold):
    accuracy = 0
    for i in range(5):
        test_data = train_folds[i].drop('label', axis=1).values
        test_labels = train_folds[i]['label'].values
        train_data = pd.concat([train_folds[j] for j in range(5) if j != i]).drop('label', axis=1).values
        train_labels = pd.concat([train_folds[j] for j in range(5) if j != i])['label'].values
        w = stochastic_sub_gradient_descent_SVM(train_data, train_labels, LR_0, C, T, SVM_obj_threshold)
        N = len(test_data)
        correct = 0
        for i in range(N):
            # this is where we need to do the false/true positive/negative stuff
            prediction = np.dot(w, test_data[i])
            actual = test_labels[i] 
            if prediction > 0 and actual == 1:
                correct += 1
            elif prediction < 0 and actual == -1:
                correct += 1
        accuracy += correct/N
            
    return accuracy/5

In [8]:
def eval_guesses_to_csv(w, data, filename):
    examples = []
    guesses = []
    i = 0
    # majority_label = dtID3.determine_majority_label(data)
    for row in data.iterrows():
        examples.append(i)
        # print(row[1].shape)
        # print(a.shape)
        # print(a)
        guessLabel = np.dot(w, row[1].values)
        if guessLabel > 0:
            guesses.append(1)
        else:
            guesses.append(0)
        i += 1
    examples_df = pd.DataFrame(examples, columns=['example_id'])
    guesses_df = pd.DataFrame(guesses, columns=['label'])
    guesses_df = pd.concat([examples_df, guesses_df], axis=1)
    guesses_df.to_csv(filename, index=False)

---------- GLOVE ------------

In [9]:
glove_train_smol = glove_train.iloc[:100]
glove_train_smol_folds = np.array_split(glove_train_smol, 5)
glove_train_folds = np.array_split(glove_train, 5)

In [10]:
### testing: detemine best hyperparameters 

T = 100 # we probs (shouldn't) reach this, cause it should stop on its own cause we'll converge to the bottom of the bowl
SVM_obj_threshold = 0.001
LR_0_arr = [1, .1, .01, .001, .0001]
C_arr = [10, 1, .1, .01, .001, .0001]

bestHyperparameters = {"LR_0": 0, "C": 0, "avg_accuracy": 0}

train_folds = glove_train_folds
for LR_0 in LR_0_arr:
    for C in C_arr:
        avg_accuracy = CV(train_folds, C, LR_0, T, SVM_obj_threshold)
        print(f'LR_0 = {LR_0}, C = {C}, avg_accuracy = {avg_accuracy}')
        if avg_accuracy > bestHyperparameters["avg_accuracy"]:
            bestHyperparameters["LR_0"] = LR_0
            bestHyperparameters["C"] = C
            bestHyperparameters["avg_accuracy"] = avg_accuracy
print("best Hyperparameters: ", bestHyperparameters)       
        

LR_0 = 1, C = 10, avg_accuracy = 0.49531428571428576
LR_0 = 1, C = 1, avg_accuracy = 0.49531428571428576
LR_0 = 1, C = 0.1, avg_accuracy = 0.49531428571428576
LR_0 = 1, C = 0.01, avg_accuracy = 0.49531428571428576
LR_0 = 1, C = 0.001, avg_accuracy = 0.4953714285714286
LR_0 = 1, C = 0.0001, avg_accuracy = 0.4953714285714286
LR_0 = 0.1, C = 10, avg_accuracy = 0.5063428571428571
LR_0 = 0.1, C = 1, avg_accuracy = 0.5065714285714286
LR_0 = 0.1, C = 0.1, avg_accuracy = 0.5076
LR_0 = 0.1, C = 0.01, avg_accuracy = 0.4958857142857143
LR_0 = 0.1, C = 0.001, avg_accuracy = 0.5060571428571429
LR_0 = 0.1, C = 0.0001, avg_accuracy = 0.502
LR_0 = 0.01, C = 10, avg_accuracy = 0.5057142857142858
LR_0 = 0.01, C = 1, avg_accuracy = 0.5069714285714286
LR_0 = 0.01, C = 0.1, avg_accuracy = 0.5327999999999999
LR_0 = 0.01, C = 0.01, avg_accuracy = 0.5660571428571428
LR_0 = 0.01, C = 0.001, avg_accuracy = 0.5098857142857143
LR_0 = 0.01, C = 0.0001, avg_accuracy = 0.5157714285714287
LR_0 = 0.001, C = 10, avg_ac

In [11]:
### testing: using best hyperparameters, train model and test on test data

#####
train = glove_train
test = glove_test
#####

LR_0 = bestHyperparameters["LR_0"]
C = bestHyperparameters["C"]

train_data = train.drop('label', axis=1).values
train_labels = train['label'].values

w = stochastic_sub_gradient_descent_SVM(train_data, train_labels, LR_0, C, T, SVM_obj_threshold)

test_data = test.drop('label', axis=1).values
test_labels = test['label'].values
N = len(test_data)
correct = 0
accuracy = 0
for i in range(N):
    # this is where we need to do the false/true positive/negative stuff
    prediction = np.dot(w, test_data[i])
    actual = test_labels[i] 
    if prediction > 0 and actual == 1:
        correct += 1
    elif prediction < 0 and actual == -1:
        correct += 1
accuracy = correct/N


print("On Test Set: -------------------")
print("Accuracy: ", accuracy)


On Test Set: -------------------
Accuracy:  0.6484444444444445


In [12]:
eval_guesses_to_csv(w, glove_eval.drop(columns=['label']), "SVM.glove.eval.predictions.csv")

----------- tfidf -------------

In [13]:
tfidf_train_folds = np.array_split(tfidf_train, 5)

In [14]:
### testing: detemine best hyperparameters 

T = 100 # we probs (shouldn't) reach this, cause it should stop on its own cause we'll converge to the bottom of the bowl
SVM_obj_threshold = 0.001
LR_0_arr = [1, .1, .01, .001, .0001]
C_arr = [10, 1, .1, .01, .001, .0001]

bestHyperparameters = {"LR_0": 0, "C": 0, "avg_accuracy": 0}

train_folds = tfidf_train_folds

for LR_0 in LR_0_arr:
    for C in C_arr:
        avg_accuracy = CV(train_folds, C, LR_0, T, SVM_obj_threshold)
        print(f'LR_0 = {LR_0}, C = {C}, avg_accuracy = {avg_accuracy}')
        if avg_accuracy > bestHyperparameters["avg_accuracy"]:
            bestHyperparameters["LR_0"] = LR_0
            bestHyperparameters["C"] = C
            bestHyperparameters["avg_accuracy"] = avg_accuracy
print("best Hyperparameters: ", bestHyperparameters)       
        

LR_0 = 1, C = 10, avg_accuracy = 0.5493714285714285
LR_0 = 1, C = 1, avg_accuracy = 0.5091428571428571
LR_0 = 1, C = 0.1, avg_accuracy = 0.5091428571428571
LR_0 = 1, C = 0.01, avg_accuracy = 0.5091428571428571
LR_0 = 1, C = 0.001, avg_accuracy = 0.5091428571428571
LR_0 = 1, C = 0.0001, avg_accuracy = 0.5091428571428571
LR_0 = 0.1, C = 10, avg_accuracy = 0.5792
LR_0 = 0.1, C = 1, avg_accuracy = 0.5795428571428572
LR_0 = 0.1, C = 0.1, avg_accuracy = 0.5795428571428572
LR_0 = 0.1, C = 0.01, avg_accuracy = 0.5795428571428572
LR_0 = 0.1, C = 0.001, avg_accuracy = 0.5795428571428572
LR_0 = 0.1, C = 0.0001, avg_accuracy = 0.5790285714285714
LR_0 = 0.01, C = 10, avg_accuracy = 0.6344
LR_0 = 0.01, C = 1, avg_accuracy = 0.6344
LR_0 = 0.01, C = 0.1, avg_accuracy = 0.6344
LR_0 = 0.01, C = 0.01, avg_accuracy = 0.6344
LR_0 = 0.01, C = 0.001, avg_accuracy = 0.6344
LR_0 = 0.01, C = 0.0001, avg_accuracy = 0.6344
LR_0 = 0.001, C = 10, avg_accuracy = 0.6809714285714286
LR_0 = 0.001, C = 1, avg_accuracy =

In [15]:
### testing: using best hyperparameters, train model and test on test data

#####
train = tfidf_train
test = tfidf_test
#####

LR_0 = bestHyperparameters["LR_0"]
C = bestHyperparameters["C"]

train_data = train.drop('label', axis=1).values
train_labels = train['label'].values

w = stochastic_sub_gradient_descent_SVM(train_data, train_labels, LR_0, C, T, SVM_obj_threshold)

test_data = test.drop('label', axis=1).values
test_labels = test['label'].values
N = len(test_data)
correct = 0
accuracy = 0
for i in range(N):
    # this is where we need to do the false/true positive/negative stuff
    prediction = np.dot(w, test_data[i])
    actual = test_labels[i] 
    if prediction > 0 and actual == 1:
        correct += 1
    elif prediction < 0 and actual == -1:
        correct += 1
accuracy = correct/N


print("On Test Set: -------------------")
print("Accuracy: ", accuracy)


On Test Set: -------------------
Accuracy:  0.688


In [16]:
eval_guesses_to_csv(w, tfidf_eval.drop(columns=['label']), "SVM.tfidf.eval.predictions.csv")

---------- BOW ------------

In [17]:
BOW_train_folds = np.array_split(BOW_train, 5)

In [18]:
### testing: detemine best hyperparameters 

T = 100 # we probs (shouldn't) reach this, cause it should stop on its own cause we'll converge to the bottom of the bowl
SVM_obj_threshold = 0.001
LR_0_arr = [1, .1, .01, .001, .0001]
C_arr = [10, 1, .1, .01, .001, .0001]

bestHyperparameters = {"LR_0": 0, "C": 0, "avg_accuracy": 0}

train_folds = BOW_train_folds

for LR_0 in LR_0_arr:
    for C in C_arr:
        avg_accuracy = CV(train_folds, C, LR_0, T, SVM_obj_threshold)
        print(f'LR_0 = {LR_0}, C = {C}, avg_accuracy = {avg_accuracy}')
        if avg_accuracy > bestHyperparameters["avg_accuracy"]:
            bestHyperparameters["LR_0"] = LR_0
            bestHyperparameters["C"] = C
            bestHyperparameters["avg_accuracy"] = avg_accuracy
print("best Hyperparameters: ", bestHyperparameters)       
        

LR_0 = 1, C = 10, avg_accuracy = 0.5051428571428571
LR_0 = 1, C = 1, avg_accuracy = 0.5089714285714285
LR_0 = 1, C = 0.1, avg_accuracy = 0.5047428571428572
LR_0 = 1, C = 0.01, avg_accuracy = 0.5030857142857144
LR_0 = 1, C = 0.001, avg_accuracy = 0.5030285714285715
LR_0 = 1, C = 0.0001, avg_accuracy = 0.5031428571428572
LR_0 = 0.1, C = 10, avg_accuracy = 0.5584
LR_0 = 0.1, C = 1, avg_accuracy = 0.5409142857142857
LR_0 = 0.1, C = 0.1, avg_accuracy = 0.5689714285714286
LR_0 = 0.1, C = 0.01, avg_accuracy = 0.5445142857142857
LR_0 = 0.1, C = 0.001, avg_accuracy = 0.5125714285714287
LR_0 = 0.1, C = 0.0001, avg_accuracy = 0.5125714285714287
LR_0 = 0.01, C = 10, avg_accuracy = 0.6089142857142857
LR_0 = 0.01, C = 1, avg_accuracy = 0.6270857142857142
LR_0 = 0.01, C = 0.1, avg_accuracy = 0.5703428571428572
LR_0 = 0.01, C = 0.01, avg_accuracy = 0.5562285714285714
LR_0 = 0.01, C = 0.001, avg_accuracy = 0.5226285714285714
LR_0 = 0.01, C = 0.0001, avg_accuracy = 0.5226285714285714
LR_0 = 0.001, C = 1

In [19]:
### testing: using best hyperparameters, train model and test on test data

#####
train = BOW_train
test = BOW_test
#####

LR_0 = bestHyperparameters["LR_0"]
C = bestHyperparameters["C"]

train_data = train.drop('label', axis=1).values
train_labels = train['label'].values

w = stochastic_sub_gradient_descent_SVM(train_data, train_labels, LR_0, C, T, SVM_obj_threshold)

test_data = test.drop('label', axis=1).values
test_labels = test['label'].values
N = len(test_data)
correct = 0
accuracy = 0
for i in range(N):
    # this is where we need to do the false/true positive/negative stuff
    prediction = np.dot(w, test_data[i])
    actual = test_labels[i] 
    if prediction > 0 and actual == 1:
        correct += 1
    elif prediction < 0 and actual == -1:
        correct += 1
accuracy = correct/N


print("On Test Set: -------------------")
print("Accuracy: ", accuracy)


On Test Set: -------------------
Accuracy:  0.7075555555555556


In [20]:
eval_guesses_to_csv(w, BOW_eval.drop(columns=['label']), "SVM.BOW.eval.predictions.csv")